# Append LGA Codes

- Processes `pandas dataframe` containing local government area names, attempts to clean the names and produce new code/names based on Australian Bureau of Statistics naming conventions.
- Data can be import/exported as excel/csv/sql etc

### Import modules

In [1]:
import datetime
import pandas
import numpy
from multiprocessing import Pool, cpu_count

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Import data

In [2]:
import_path = '/users/danielcorcoran/desktop/DirtyLgas.xlsx'
export_path = '/users/danielcorcoran/desktop/'

data = pandas.read_excel(import_path, sheet_name = 'Sheet1')
data.head()

,Local Government Area,Suburb,Some Metric 1,Some Metric 2,Some Metric 3
0,Ballarat (C),Ballarat,119,93,30
1,statewide,statewide,11,60,81
2,statewide,statewide,67,110,51
3,Melbourne (C),Parkville,37,2,51
4,statewide,statewide,66,70,42


### Set Lga Column Name
- This variable may change depending on your dataset

In [3]:
lga_column_name = 'Local Government Area'

### Function to clean dirty lga string 
**Operations**
- Uppercase.
- Strip.
- Replace anything which is not A-Z with spaces.
- Replace all double spaces with single spaces.

In [4]:
lga_dict = {20110: ('ALPINE','Alpine (S)')
    ,20260: ('ARARAT', 'Ararat (RC)')
    ,20570: ('BALLARAT', 'Ballarat (C)')}

In [5]:
def clean_data(dirty_lga):
    
    lga_dict = {20110: ('ALPINE','Alpine (S)')
    ,20260: ('ARARAT', 'Ararat (RC)')
    ,20570: ('BALLARAT', 'Ballarat (C)')
    ,20660: ('BANYULE', 'Banyule (C)')
    ,20740: ('BASS COAST','Bass Coast (S)')
    ,20830: ('BAW BAW', 'Baw Baw (S)')
    ,20910: ('BAYSIDE', 'Bayside (C)')
    ,21010: ('BENALLA', 'Benalla (RC)')
    ,21110: ('BOROONDARA', 'Boroondara (C)')
    ,21180: ('BRIMBANK', 'Brimbank (C)')
    ,21270: ('BULOKE', 'Buloke (S)')
    ,21370: ('CAMPASPE', 'Campaspe (S)')
    ,21450: ('CARDINIA', 'Cardinia (S)')
    ,21610: ('CASEY', 'Casey (C)')
    ,21670: ('CENTRAL GOLDFIELDS', 'Central Goldfields (S)')
    ,21750: ('COLAC OTWAY', 'Colac-Otway (S)')
    ,21830: ('CORANGAMITE', 'Corangamite (S)')
    ,21890: ('DAREBIN', 'Darebin (C)')
    ,22110: ('EAST GIPPSLAND', 'East Gippsland (S)')
    ,22170: ('FRANKSTON', 'Frankston (C)')
    ,22250: ('GANNAWARRA', 'Gannawarra (S)')
    ,22310: ('GLEN EIRA', 'Glen Eira (C)')
    ,22410: ('GLENELG', 'Glenelg (S)')
    ,22490: ('GOLDEN PLAINS', 'Golden Plains (S)')
    ,22620: ('GREATER BENDIGO', 'Greater Bendigo (C)')
    ,22670: ('GREATER DANDENONG', 'Greater Dandenong (C)')
    ,22750: ('GREATER GEELONG', 'Greater Geelong (C)')
    ,22830: ('GREATER SHEPPARTON', 'Greater Shepparton (C)')
    ,22910: ('HEPBURN', 'Hepburn (S)')
    ,22980: ('HINDMARSH', 'Hindmarsh (S)')
    ,23110: ('HOBSONS BAY', 'Hobsons Bay (C)')
    ,23190: ('HORSHAM', 'Horsham (RC)')
    ,23270: ('HUME', 'Hume (C)')
    ,23350: ('INDIGO', 'Indigo (S)')
    ,23430: ('KINGSTON', 'Kingston (C)(Vic.)')
    ,23670: ('KNOX', 'Knox (C)')
    ,23810: ('LATROBE', 'Latrobe (C)(Vic.)')
    ,23940: ('LODDON', 'Loddon (S)')
    ,24130: ('MACEDON RANGES', 'Macedon Ranges (S)')
    ,24210: ('MANNINGHAM', 'Manningham (C')
    ,24250: ('MANSFIELD', 'Mansfield (S)')
    ,24330: ('MARIBYRNONG', 'Maribyrnong (C)')
    ,24410: ('MAROONDAH', 'Maroondah (C)')
    ,24600: ('MELBOURNE', 'Melbourne (C)')
    ,24650: ('MELTON', 'Melton (C)')
    ,24780: ('MILDURA', 'Mildura (RC)')
    ,24850: ('MITCHELL', 'Mitchell (S)')
    ,24900: ('MOIRA', 'Moira (S)')
    ,24970: ('MONASH', 'Monash (C)')
    ,25060: ('MOONEE VALLEY', 'Moonee Valley (C)')
    ,25150: ('MOORABOOL', 'Moorabool (S)')
    ,25250: ('MORELAND', 'Moreland (C)')
    ,25340: ('MORNINGTON PENINSULA', 'Mornington Peninsula (S)')
    ,25430: ('MOUNT ALEXANDER', 'Mount Alexander (S)')
    ,25490: ('MOYNE', 'Moyne (S)')
    ,25620: ('MURRINDINDI', 'Murrindindi (S)')
    ,25710: ('NILLUMBIK', 'Nillumbik (S)')
    ,25810: ('NORTHERN GRAMPIANS', 'Northern Grampians (S)')
    ,25900: ('PORT PHILLIP', 'Port Phillip (C)')
    ,25990: ('PYRENEES', 'Pyrenees (S)')
    ,26080: ('QUEENSCLIFFE', 'Queenscliffe (B)')
    ,26170: ('SOUTH GIPPSLAND', 'South Gippsland (S)')
    ,26260: ('SOUTHERN GRAMPIANS', 'Southern Grampians (S)')
    ,26350: ('STONNINGTON', 'Stonnington (C)')
    ,26430: ('STRATHBOGIE', 'Strathbogie (S)')
    ,26490: ('SURF COAST', 'Surf Coast (S)')
    ,26610: ('SWAN HILL', 'Swan Hill (RC)')
    ,26670: ('TOWONG', 'Towong (S)')
    ,29399: ('UNINCORPORATED VIC', 'Unincorporated Vic')
    ,26700: ('WANGARATTA', 'Wangaratta (RC)')
    ,26730: ('WARRNAMBOOL', 'Warrnambool (C)')
    ,26810: ('WELLINGTON', 'Wellington (S)')
    ,26890: ('WEST WIMMERA', 'West Wimmera (S)')
    ,26980: ('WHITEHORSE', 'Whitehorse (C)')
    ,27070: ('WHITTLESEA', 'Whittlesea (C)')
    ,27170: ('WODONGA', 'Wodonga (C)')
    ,27260: ('WYNDHAM', 'Wyndham (C)')
    ,27450: ('YARRA RANGES', 'Yarra Ranges (S)')
    ,27350: ('YARRA', 'Yarra (C)')
    ,27630: ('YARRIAMBIACK', 'Yarriambiack (S)')
    ,29499: ('NO USUAL ADDRESS', 'No usual address (Vic.)')
    ,29799: ('MIGRATORY OFFSHORE SHIPPING', 'Migratory - Offshore - Shipping (Vic.)')}
        
    dirty_lga_upper = str(dirty_lga).upper()
    string = dirty_lga_upper
    
    acceptable_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    
    for index, char in enumerate(string):
        if char not in acceptable_chars:
            string = string.replace(char, " ")

    dirty_lga_upper = string

    while "  " in dirty_lga_upper:
        dirty_lga_upper = dirty_lga_upper.replace("  ", " ")

    cleaned_lga_upper = dirty_lga_upper.strip()

    for key in lga_dict.keys():
        reference_code = key
        reference_name = lga_dict[key][0]
        lga_abs_name = lga_dict[key][1]

        if reference_name in cleaned_lga_upper:

            return (reference_code, lga_abs_name)

### Process and time

In [7]:
%%time
results = data[lga_column_name].apply(clean_data).apply(pandas.Series)
results.columns = ['LGA_CODE16_SCRIPT', 'LGA_NAME16_SCRIPT']
final_data = pandas.concat([data, results], axis = 1)

CPU times: user 21.7 s, sys: 354 ms, total: 22.1 s
Wall time: 22 s


In [8]:
final_data.head()

,Local Government Area,Suburb,Some Metric 1,Some Metric 2,Some Metric 3,LGA_CODE16_SCRIPT,LGA_NAME16_SCRIPT
0,Ballarat (C),Ballarat,119,93,30,20570.0,Ballarat (C)
1,statewide,statewide,11,60,81,NaN,NaN
2,statewide,statewide,67,110,51,NaN,NaN
3,Melbourne (C),Parkville,37,2,51,24600.0,Melbourne (C)
4,statewide,statewide,66,70,42,NaN,NaN


### Export Results

In [9]:
final_data.to_csv(export_path + 'CleanedLgas.csv', index = False)